# Importazione librerie

In [0]:
import scipy as sci
import numpy as np
import os
import matplotlib.pyplot as plt

import  ImageFilter as filter

# Visualizzazione statistiche audio

In [ ]:
def getStatsAudio(pathAudios):
    file_list_os = os.listdir(pathAudios)
    nFile = len(file_list_os)

    minLength = 0.25
    maxLength = 3.4
    file_list_os = os.listdir(pathAudios)
    listSamples = []
    max = 0
    min = 0
    sum = 0
    indexMax = 0
    indexMin = 0

    for i in range(0,nFile):
        nameFile = file_list_os[i]
        filePath = os.path.join(pathAudios, nameFile)
        sample_rate, samples = sci.io.wavfile.read(filePath)


        if len(samples) < sample_rate*minLength:
            print(nameFile + ' is too short ' + str(float(len(samples)/sample_rate)))
            continue

        if len(samples) > sample_rate*maxLength:
            print(nameFile + ' is too long ' + str(float(len(samples)/sample_rate)))
            continue

        print(sample_rate, len(samples), str(float(len(samples)/sample_rate)), nameFile)

        listSamples.append(samples)
        app = len(samples)

        if app>max :
            max = app
            indexMax = i
        if app<min or min==0 :
            min = app
            indexMin = i
        sum += app


    print('\n')
    print(str(len(listSamples)) + ' audios selected on ' + str(nFile))
    print('N. ' + str(indexMax+1) + ' is longest wav: ' + str(float(max)/sample_rate) + ' seconds, file: ' + file_list_os[indexMax])
    print('N. ' + str(indexMin+1) + ' is shortest wav: ' + str(float(min)/sample_rate) + ' seconds, file: ' + file_list_os[indexMin])
    print('Mean: ' + str(float(sum)/(sample_rate*len(listSamples))) + ' seconds')


# Funzioni per creazione spettrogrammi

In [2]:
#Funzione per creazione di un singolo spettrogramma
#Frequenze tra 5 kHz e 25 kHz
#Hanning window, window_size=2048, overlap 50%, nfft=2048
def getAndSaveSpectrogram(signal, fs, outputPath, window_size=2048, overlap=1024):
    #fs, signal = sci.io.wavfile.read(filename)
    step = window_size - overlap
    bins = np.arange(0, len(signal) - window_size + 1, step)
    window = np.hanning(window_size)
    spectrogram = []

    for start in bins:
        segment = signal[start:start + window_size] * window
        spectrum = np.fft.fft(segment, n=2048)[:window_size // 2]
        magnitude = np.abs(spectrum)
        magnitude_db = 20 * np.log10(magnitude)
        spectrogram.append(magnitude_db)

    spectrogram =  np.array(spectrogram).T

    max_freq = 25000
    freq_bins = int(max_freq / (fs / window_size))

    min_freq = 5000
    freq_bins_min = int(min_freq / (fs / window_size))

    # Plot the spectrogram
    plt.gray()
    plt.rcParams['axes.grid'] = False
    plt.rcParams['image.origin'] = 'lower'
    plt.rcParams['image.aspect'] = 'auto'
    plt.axis('off')

    plt.imshow(spectrogram[freq_bins_min:freq_bins, :],  extent=[0, len(signal)/fs, min_freq, max_freq])
    plt.savefig(outputPath, bbox_inches='tight', pad_inches=0)
    figure = plt.gcf()
    plt.clf()
    return figure





#Creazione degli spettrogrammi in scala di grigio da file Wav all'interno di una cartella.
#Gli audio sotto i 250 ms non vengono presi in considerazione
#Gli audio con durata superiore ad un secondo, vengono suddivisi in frame da 600 ms scorrendo ogni 200 ms e vengono memorizzati. Inoltre, viene memorizzato anche lo spettrogramma intero, della durata di tutto l'audio
def buildSpectrograms(inputDirectory, outputDirectory):
    file_list_os = os.listdir(inputDirectory)
    nFile = len(file_list_os)
    print(nFile)

    minLength = 0.25
    splitMinLength = 1
    frameStep = 0.2

    for i in range(0,nFile):
        nameFile = file_list_os[i]
        inputPath = os.path.join(inputDirectory, nameFile)

        sample_rate, samples = sci.io.wavfile.read(inputPath)
        print(sample_rate, len(samples), nameFile)

        if len(samples) < sample_rate*minLength:
            print(nameFile + ' is too short ' + str(float(len(samples)/sample_rate)))
            continue

        if len(samples) > sample_rate*splitMinLength:
            nFrame = int(len(samples)/sample_rate/frameStep) - 2
            for k in range(0, nFrame-1):
                subSamples = samples[int(k*frameStep*sample_rate) : int((k+3)*frameStep*sample_rate)]
                outputPath = os.path.join(outputDirectory, nameFile.split('.')[0] + '_' + str(k+1) + '.png')
                getAndSaveSpectrogram(subSamples, sample_rate, outputPath)

            subSamples = samples[int((nFrame-1)*frameStep*sample_rate) : ]
            outputPath = os.path.join(outputDirectory, nameFile.split('.')[0] + '_' + str(nFrame) + '.png')
            getAndSaveSpectrogram(subSamples, sample_rate, outputPath)




        outputPath = os.path.join(outputDirectory, nameFile.split('.')[0] + '.png')
        getAndSaveSpectrogram(samples, sample_rate, outputPath)


## Creazione spettrogrammi in scala di grigio
Gli spettrogrammi dei fischi e dei rumori vengono memorizzati rispettivamente nelle cartelle Whistle e Noise

In [ ]:
buildSpectrograms('Data\\Audio\\Whistle', 'Data\\SpectrogramGray\\Whistle')
buildSpectrograms('Data\\Audio\\Noise', 'Data\\SpectrogramGray\\Noise')

# Applicazione filtro Sobel
Tramite il codice scritto nel file ImageFilter.py, viene applicato il filtro Sobel alle immagini presenti nella cartella dei fischi e dei rumori.
Il filtro Sobel viene utilizzato verticalmente per eliminare il rumore di sottofondo e i rumori verticali, evidenziando le caratteristiche orizzontali dell'immagine.

In [ ]:
filter.sobelFilterDirectory('Data\\SpectrogramGray\\Whistle', 'Data\\Sobel\\Whistle')
filter.sobelFilterDirectory('Data\\SpectrogramGray\\Noise', 'Data\\Sobel\\Noise')